# args

In [45]:
# from google.colab import drive
# drive.mount('/content/drive')
# !pip install transformers
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import argparse
import datetime
import re
import time
import wandb
import torch
import numpy as np
import random
# import wandb
import yaml
from transformers import AdamW, get_cosine_schedule_with_warmup
from tqdm.auto import tqdm
# from path import Path
from sklearn.metrics import mean_absolute_error, f1_score, accuracy_score


parser = argparse.ArgumentParser()
parser.add_argument('--learning_rate', default=2e-5, type=float, help='Learning Rate')
parser.add_argument('--warmup_proportion', default=0.1, type=float, help='Set Warmup Proportion')
parser.add_argument('--num_epochs', default=80, type=int, help='Set Number of Epochs')
parser.add_argument('--batch_size', default=1, type=int, help='Set Batch Size')
parser.add_argument('--max_text_length', default=150, type=int, help='Set Max Text Length')
parser.add_argument('--max_time', default=30, type=int, help='Set Max Time')
parser.add_argument('--mode', default='train', type=str, choices=['train', 'test'], help='Train Or Test Mode')
# parser.add_argument('--config', type=str, help='Path to the config.yaml file')

# 预训练模块参数
parser.add_argument('--gnn_layers', default=4, type=int)
parser.add_argument('--hidden_dim', default=300, type=int)
parser.add_argument('--emb_dim', default=100, type=int)

args = parser.parse_args([])
print(args)

Namespace(batch_size=1, emb_dim=100, gnn_layers=4, hidden_dim=300, learning_rate=2e-05, max_text_length=150, max_time=30, mode='train', num_epochs=80, warmup_proportion=0.1)


In [46]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0' 
import transformers
import argparse
from transformers import RobertaPreTrainedModel
from transformers.models.roberta.modeling_roberta import RobertaEmbeddings, RobertaEncoder, RobertaPooler, RobertaLMHead, RobertaClassificationHead
from transformers.modeling_outputs import BaseModelOutputWithPoolingAndCrossAttentions, MaskedLMOutput, SequenceClassifierOutput
from transformers.utils import logging
import torch
import torch.nn as nn
from transformers.modeling_outputs import MaskedLMOutput
from transformers import AutoTokenizer, AutoModelForMaskedLM
import pickle
from torch.utils.data import DataLoader
import torch.nn.functional as F
logger = logging.get_logger()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# RobertaTEASEL

In [47]:

class RobertaTEASEL(RobertaPreTrainedModel):
    """

    The model can behave as an encoder (with only self-attention) as well as a decoder, in which case a layer of
    cross-attention is added between the self-attention layers, following the architecture described in `Attention is
    all you need`_ by Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz
    Kaiser and Illia Polosukhin.

    To behave as an decoder the model needs to be initialized with the :obj:`is_decoder` argument of the configuration
    set to :obj:`True`. To be used in a Seq2Seq model, the model needs to initialized with both :obj:`is_decoder`
    argument and :obj:`add_cross_attention` set to :obj:`True`; an :obj:`encoder_hidden_states` is then expected as an
    input to the forward pass.

    .. _`Attention is all you need`: https://arxiv.org/abs/1706.03762

    """

    _keys_to_ignore_on_load_missing = [r"position_ids"]

    # Copied from transformers.models.bert.modeling_bert.BertModel.__init__ with Bert->Roberta
    def __init__(self, config, add_pooling_layer=True):
        super().__init__(config)
        self.config = config

        self.embeddings = RobertaEmbeddings(config)
        self.encoder = RobertaEncoder(config)

        self.pooler = RobertaPooler(config) if add_pooling_layer else None

        self.init_weights()

    def get_input_embeddings(self):
        return self.embeddings.word_embeddings

    def set_input_embeddings(self, value):
        self.embeddings.word_embeddings = value

    def _prune_heads(self, heads_to_prune):
        """
        Prunes heads of the model. heads_to_prune: dict of {layer_num: list of heads to prune in this layer} See base
        class PreTrainedModel
        """
        for layer, heads in heads_to_prune.items():
            self.encoder.layer[layer].attention.prune_heads(heads)

    # Copied from transformers.models.bert.modeling_bert.BertModel.forward
    def forward(
        self,
        acoustic_embedding,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        past_key_values=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        r"""
        encoder_hidden_states  (:obj:`torch.FloatTensor` of shape :obj:`(batch_size, sequence_length, hidden_size)`, `optional`):
            Sequence of hidden-states at the output of the last layer of the encoder. Used in the cross-attention if
            the model is configured as a decoder.
        encoder_attention_mask (:obj:`torch.FloatTensor` of shape :obj:`(batch_size, sequence_length)`, `optional`):
            Mask to avoid performing attention on the padding token indices of the encoder input. This mask is used in
            the cross-attention if the model is configured as a decoder. Mask values selected in ``[0, 1]``:

            - 1 for tokens that are **not masked**,
            - 0 for tokens that are **masked**.
        past_key_values (:obj:`tuple(tuple(torch.FloatTensor))` of length :obj:`config.n_layers` with each tuple having 4 tensors of shape :obj:`(batch_size, num_heads, sequence_length - 1, embed_size_per_head)`):
            Contains precomputed key and value hidden states of the attention blocks. Can be used to speed up decoding.

            If :obj:`past_key_values` are used, the user can optionally input only the last :obj:`decoder_input_ids`
            (those that don't have their past key value states given to this model) of shape :obj:`(batch_size, 1)`
            instead of all :obj:`decoder_input_ids` of shape :obj:`(batch_size, sequence_length)`.
        use_cache (:obj:`bool`, `optional`):
            If set to :obj:`True`, :obj:`past_key_values` key value states are returned and can be used to speed up
            decoding (see :obj:`past_key_values`).
        """
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if self.config.is_decoder:
            use_cache = use_cache if use_cache is not None else self.config.use_cache
        else:
            use_cache = False

        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
        elif input_ids is not None:
            input_shape = input_ids.size()
        elif inputs_embeds is not None:
            input_shape = inputs_embeds.size()[:-1]
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        batch_size, seq_length = input_shape
        device = input_ids.device if input_ids is not None else inputs_embeds.device

        # past_key_values_length
        past_key_values_length = past_key_values[0][0].shape[2] if past_key_values is not None else 0

        if attention_mask is None:
            attention_mask = torch.ones(((batch_size, seq_length + past_key_values_length)), device=device)

        if token_type_ids is None:
            if hasattr(self.embeddings, "token_type_ids"):
                buffered_token_type_ids = self.embeddings.token_type_ids[:, :seq_length]
                buffered_token_type_ids_expanded = buffered_token_type_ids.expand(batch_size, seq_length)
                token_type_ids = buffered_token_type_ids_expanded
            else:
                token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=device)

        # We can provide a self-attention mask of dimensions [batch_size, from_seq_length, to_seq_length]
        # ourselves in which case we just need to make it broadcastable to all heads.
        extended_attention_mask: torch.Tensor = self.get_extended_attention_mask(attention_mask, input_shape, device)

        # If a 2D or 3D attention mask is provided for the cross-attention
        # we need to make broadcastable to [batch_size, num_heads, seq_length, seq_length]
        if self.config.is_decoder and encoder_hidden_states is not None:
            encoder_batch_size, encoder_sequence_length, _ = encoder_hidden_states.size()
            encoder_hidden_shape = (encoder_batch_size, encoder_sequence_length)
            if encoder_attention_mask is None:
                encoder_attention_mask = torch.ones(encoder_hidden_shape, device=device)
            encoder_extended_attention_mask = self.invert_attention_mask(encoder_attention_mask)
        else:
            encoder_extended_attention_mask = None

        # Prepare head mask if needed
        # 1.0 in head_mask indicate we keep the head
        # attention_probs has shape bsz x n_heads x N x N
        # input head_mask has shape [num_heads] or [num_hidden_layers x num_heads]
        # and head_mask is converted to shape [num_hidden_layers x batch x num_heads x seq_length x seq_length]
        head_mask = self.get_head_mask(head_mask, self.config.num_hidden_layers)

        embedding_output = self.embeddings(
            input_ids=input_ids,
            position_ids=position_ids,
            token_type_ids=token_type_ids,
            inputs_embeds=inputs_embeds,
            past_key_values_length=past_key_values_length,
        )
        
        # Create the fusion of acoustic and text embedding out
        # 融合文本和音频的向量  位置分别是 CLS  ca embeddings
        # print('embedding_output',embedding_output.shape)
        # print('acoustic_embedding',acoustic_embedding.shape)

        fusion_embedding = torch.cat((torch.unsqueeze(embedding_output[:,0,:],1), acoustic_embedding, embedding_output[:,1:,:]),1)
        # print('fusion_embedding',fusion_embedding.shape)
        # print('extended_attention_mask',extended_attention_mask.shape)
        
        encoder_outputs = self.encoder(
            fusion_embedding,
            attention_mask=extended_attention_mask,
            head_mask=head_mask,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_extended_attention_mask,
            past_key_values=past_key_values,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        sequence_output = encoder_outputs[0]
        pooled_output = self.pooler(sequence_output) if self.pooler is not None else None

        if not return_dict:
            return (sequence_output, pooled_output) + encoder_outputs[1:]

        return BaseModelOutputWithPoolingAndCrossAttentions(
            last_hidden_state=sequence_output,
            pooler_output=pooled_output,
            past_key_values=encoder_outputs.past_key_values,
            hidden_states=encoder_outputs.hidden_states,
            attentions=encoder_outputs.attentions,
            cross_attentions=encoder_outputs.cross_attentions,
        )


# RobertaForMaskedLM_Teasel

In [48]:

class RobertaForMaskedLM_Teasel(RobertaPreTrainedModel):
    _keys_to_ignore_on_save = [r"lm_head.decoder.weight", r"lm_head.decoder.bias"]
    _keys_to_ignore_on_load_missing = [r"position_ids", r"lm_head.decoder.weight", r"lm_head.decoder.bias"]
    _keys_to_ignore_on_load_unexpected = [r"pooler"]

    def __init__(self, config):
        super().__init__(config)

        if config.is_decoder:
            logger.warning(
                "If you want to use `RobertaForMaskedLM` make sure `config.is_decoder=False` for "
                "bi-directional self-attention."
            )

        self.roberta = RobertaTEASEL(config, add_pooling_layer=True)
        self.lm_head = RobertaLMHead(config)

        # The LM head weights require special treatment only when they are tied with the word embeddings
        self.update_keys_to_ignore(config, ["lm_head.decoder.weight"])

        self.init_weights()

    def get_output_embeddings(self):
        return self.lm_head.decoder

    def set_output_embeddings(self, new_embeddings):
        self.lm_head.decoder = new_embeddings

    def forward(
        self,
        acoustic_embedding,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size, sequence_length)`, `optional`):
            Labels for computing the masked language modeling loss. Indices should be in ``[-100, 0, ...,
            config.vocab_size]`` (see ``input_ids`` docstring) Tokens with indices set to ``-100`` are ignored
            (masked), the loss is only computed for the tokens with labels in ``[0, ..., config.vocab_size]``
        kwargs (:obj:`Dict[str, any]`, optional, defaults to `{}`):
            Used to hide legacy arguments that have been deprecated.
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.roberta(
            acoustic_embedding,
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        sequence_output = outputs[0]
        prediction_scores = self.lm_head(sequence_output)

        masked_lm_loss = None
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            masked_lm_loss = loss_fct(prediction_scores.view(-1, self.config.vocab_size), labels.view(-1))

        if not return_dict:
            output = (prediction_scores,) + outputs[2:]
            return ((masked_lm_loss,) + output) if masked_lm_loss is not None else output

        return MaskedLMOutput(
            loss=masked_lm_loss,
            logits=prediction_scores,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


# RobertaForSequenceClassification

In [49]:

class RobertaForSequenceClassification(RobertaPreTrainedModel):
    _keys_to_ignore_on_load_missing = [r"position_ids"]

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config

        self.roberta = RobertaTEASEL(config, add_pooling_layer=False)  # 作者改造了这个模型的输入，加入一个声学的嵌入
        self.classifier = RobertaClassificationHead(config)

        self.init_weights()

    def forward(
        self,
        acoustic_embedding, # 作者改造了这个模型的输入，加入一个声学的嵌入
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`):
            Labels for computing the sequence classification/regression loss. Indices should be in :obj:`[0, ...,
            config.num_labels - 1]`. If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.roberta(
            acoustic_embedding,
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        sequence_output = outputs[0]
        logits = self.classifier(sequence_output)
        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = torch.nn.MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze())
                else:
                    loss = loss_fct(logits, labels)
            elif self.config.problem_type == "single_label_classification":
                # c_num = [1574,6583,2529,1032,3532,100,750]
                # total = 16293
                # loss_weights = torch.FloatTensor([total / i / 2 for i in c_num])
                # loss_weights = torch.FloatTensor([1, 0.5, 1, 1.8, 1, 4, 1.5])
                # loss_fct = torch.nn.CrossEntropyLoss(loss_weights.to('cuda'))
                loss_fct = torch.nn.CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = torch.nn.BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )



# GATgather

In [50]:
# GATgather 网络会用到的层
class GAT_dialoggcn_v1(torch.nn.Module):
    '''
    use linear to avoid OOM
    H_i = alpha_ij(W_rH_j)
    alpha_ij = attention(H_i, H_j)
    '''
    def __init__(self, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.linear = nn.Linear(hidden_size * 2, 1)
        self.Wr0 = nn.Linear(hidden_size, hidden_size, bias = False)
        self.Wr1 = nn.Linear(hidden_size, hidden_size, bias = False)

    def forward(self, Q, K, V, adj, s_mask):
        '''
        imformation gatherer with linear attention
        :param Q: (B, D) # query utterance
        :param K: (B, N, D) # context
        :param V: (B, N, D) # context
        :param adj: (B,  N) # the adj matrix of the i th node
        :param s_mask: (B,  N) #
        :return:
        '''

        B = K.size()[0]
        N = K.size()[1]
        Q = Q.unsqueeze(1).expand(-1, N, -1) # (B, N, D)；
        X = torch.cat((Q,K), dim = 2) # (B, N, 2D)
        alpha = self.linear(X).permute(0,2,1) #(B, 1, N)
        adj = adj.unsqueeze(1)  # (B, 1, N)
        alpha = self.mask_logic(alpha, adj) # (B, 1, N)
        attn_weight = F.softmax(alpha, dim = 2) # (B, 1, N)
        V0 = self.Wr0(V) # (B, N, D)
        V1 = self.Wr1(V) # (B, N, D)
        s_mask = s_mask.unsqueeze(2).float()   # (B, N, 1)
        V = V0 * s_mask + V1 * (1 - s_mask) # 一个控制相同说话人的特征 一个控制不同说话人的特征
        attn_sum = torch.bmm(attn_weight, V).squeeze(1) # (B, D) # 注意力的权重乘以特征
        return attn_weight, attn_sum
    
    def mask_logic(self, alpha, adj):
        '''
        performing mask logic with adj
        :param alpha:
        :param adj:
        :return:
        '''
        return alpha - (1 - adj) * 1e30
    
class attentive_node_features(nn.Module):
    '''
    Method to obtain attentive node features over the graph convoluted features
    '''
    def __init__(self, hidden_size):
        super().__init__()
        self.transform = nn.Linear(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, 768) # 最后调整成768


    def forward(self,features, lengths, nodal_att_type):
        '''
        features : (B, N, V)
        lengths : (B, )
        nodal_att_type : type of the final nodal attention
        '''

        if nodal_att_type==None:
            return features

        batch_size = features.size(0)
        max_seq_len = features.size(1)
        padding_mask = [l*[1]+(max_seq_len-l)*[0] for l in lengths]
        padding_mask = torch.tensor(padding_mask).to(features)    # (B, N)
        causal_mask = torch.ones(max_seq_len, max_seq_len).to(features)  # (N, N)
        causal_mask = torch.tril(causal_mask).unsqueeze(0)  # (1, N, N)

        if nodal_att_type=='global':
            mask = padding_mask.unsqueeze(1)
        elif nodal_att_type=='past':
            mask = padding_mask.unsqueeze(1)*causal_mask

        x = self.transform(features)  # (B, N, V)
        temp = torch.bmm(x, features.permute(0,2,1))
        #print(temp)
        alpha = F.softmax(torch.tanh(temp), dim=2)  # (B, N, N)
        alpha_masked = alpha*mask  # (B, N, N)
        
        alpha_sum = torch.sum(alpha_masked, dim=2, keepdim=True)  # (B, N, 1)
        #print(alpha_sum)
        alpha = alpha_masked / alpha_sum    # (B, N, N)
        attn_pool = torch.bmm(alpha, features)  # (B, N, V)
        attn_pool = self.fc(attn_pool)
        return attn_pool
    
# 将要被预训练的网络 用于聚合上下文信息 作为文本的提示
class GATgather(torch.nn.Module):
   # 输入 一个对话 的 句子级特征 1 seq feature_size
   
   # 该对话的 邻接矩阵 
   # 掩码矩阵
    def __init__(self, device='cpu', args=None):
        super().__init__()
        gats = []
        self.args = args
        for _ in range(args.gnn_layers):
            gats += [GAT_dialoggcn_v1(args.hidden_dim)]
        self.gather = nn.ModuleList(gats)
        grus_c = []
        for _ in range(args.gnn_layers):
            grus_c += [nn.GRUCell(args.hidden_dim, args.hidden_dim)]
        self.grus_c = nn.ModuleList(grus_c)

        grus_p = []
        for _ in range(args.gnn_layers):
            grus_p += [nn.GRUCell(args.hidden_dim, args.hidden_dim)]
        self.grus_p = nn.ModuleList(grus_p)
        
        self.fc1 = nn.Linear(args.emb_dim, args.hidden_dim)
        
        in_dim = args.hidden_dim * (args.gnn_layers + 1) + args.emb_dim
        self.attentive_node_features = attentive_node_features(in_dim)
        self.dropout = nn.Dropout(0.2)
        self.nodal_att_type = 'past'
    def forward(self, features, adj, s_mask, lengths):
        '''
        :param features: (B, N, D)
        :param adj: (B, N, N)
        :param s_mask: (B, N, N)
        :return:
        '''
        num_utter = features.size()[1]
        H0 = F.relu(self.fc1(features))
        # H0 = self.dropout(H0)
        H = [H0]
        for l in range(self.args.gnn_layers):
            C = self.grus_c[l](H[l][:,0,:]).unsqueeze(1) 
            M = torch.zeros_like(C).squeeze(1) 
            P = self.grus_p[l](M, H[l][:,0,:]).unsqueeze(1)  
            H1 = C+P
            # print('H1',H1.shape)
            for i in range(1, num_utter):
              _, M = self.gather[l](H[l][:,i,:], H1, H1, adj[:,i,:i], s_mask[:,i,:i])                  
              C = self.grus_c[l](H[l][:,i,:], M).unsqueeze(1)
              P = self.grus_p[l](M, H[l][:,i,:]).unsqueeze(1) 
              C = self.dropout(C)
              P = self.dropout(P)  
              H_temp = C+P
              H1 = torch.cat((H1 , H_temp), dim = 1)  
            H.append(H1)
        H.append(features)
        H = torch.cat(H, dim = 2) 
        H = self.attentive_node_features(H,lengths,self.nodal_att_type) 
        return H # B N V  V 是 in_dim
# features = torch.rand((16, 10, 300))
# adj = torch.rand((16,10,10))
# smask = torch.rand((16,10,10))
# lengths = torch.rand((16 ,10))
# parser = argparse.ArgumentParser()

# parser.add_argument('--gnn_layers', default=4, type=int)
# parser.add_argument('--hidden_dim', default=300, type=int)
# parser.add_argument('--emb_dim', default=300, type=int)

# args = parser.parse_args([])
# print(args)
# model = GATgather(args=args)
# model(features,adj,smask, lengths)

# robertGatPretain 

In [51]:
class robertGatPretain(torch.nn.Module):
    def __init__(self, device='cpu', args=None):
        super().__init__()
        self.device = device
        roberta_config = transformers.RobertaConfig()
        roberta_config.max_position_embeddings = 512
        roberta_config.vocab_size = 21128
        roberta_config.type_vocab_size = 2
        label2idx = {'happy':0, 'grateful':1, 'relaxed':2, 'positive-other':3, 'neutral':4,'anger':5, 'sadness':6, 'fear':7, 'depress':8, 'disgust':9, 'astonished':10, 'worried': 11, 'negative-other':12}
        id2label = {y:x for x,y in label2idx.items()}
        roberta_config.label2id  = label2idx
        roberta_config.id2label = id2label
        roberta_config.num_labels = len(label2idx)

        self.roberta_model = RobertaForSequenceClassification.from_pretrained('hfl/chinese-roberta-wwm-ext', config=roberta_config).to(device)
        self.context_gather = GATgather(args=args).to(device)
        #Freeze Layers
        for parameter in self.roberta_model.parameters():
            parameter.requires_grad = False
            
        self.loss_fct = torch.nn.CrossEntropyLoss()
    def forward(self, masked_ids, attention_mask, textf, adj, s_mask, lengths, labels):
        attention_mask = attention_mask.squeeze(0) #attention_mask多个句子就已经是批量了
        context = self.context_gather(textf, adj, s_mask, lengths)
        ca = context.squeeze(0).unsqueeze(1) # 4 1 768
         # 预训练的过程 任务是 掩码语言模型
        ## <Create the input to maskedlm model> even adjust the attention_mask
        attention_mask = torch.cat((torch.unsqueeze(attention_mask[:, 0],1), torch.ones((ca.shape[0],ca.shape[1]), device=self.device), attention_mask[:, 1:]), 1).type(torch.long)
        token_type_ids = torch.zeros(masked_ids.shape, dtype=torch.long, device=self.device)
        output = self.roberta_model(ca, masked_ids, attention_mask=attention_mask, token_type_ids=token_type_ids,labels=labels)
        return output
        

# robertGatFinetune

In [52]:
class robertGatFinetune(torch.nn.Module):
    def __init__(self, device='cpu', args=None):
        super().__init__()
        self.device = device
        roberta_config = transformers.RobertaConfig()
        roberta_config.max_position_embeddings = 514
        # roberta_config.vocab_size = 21128
        roberta_config.vocab_size = 50265
        roberta_config.type_vocab_size = 1
        label2idx ={'Happy':0, 'Neutral':1, 'Sad':2, 'Disgust':3, 'Anger': 4, 'Fear': 5, 'Surprise':6}
        id2label = {y:x for x,y in label2idx.items()}
        roberta_config.label2id  = label2idx
        roberta_config.id2label = id2label
        roberta_config.num_labels = len(label2idx)
        weights_file = './savemodel/best_pretrain_gat_1e-4.cpt'
        self.roberta_model = RobertaForSequenceClassification.from_pretrained('princeton-nlp/sup-simcse-roberta-base', config=roberta_config).to(device)
        # self.roberta_model = RobertaForSequenceClassification.from_pretrained('cyclone/simcse-chinese-roberta-wwm-ext', config=roberta_config).to(device)
        
        
        self.context_gather = GATgather(args=args).to(device)
        # Freeze Layers
        for parameter in self.roberta_model.parameters():
            parameter.requires_grad = False
        self.audio_proj = nn.Linear(100, 768).to(device)    
        self.loss_fct = torch.nn.CrossEntropyLoss()
        # self.reload_weights(weights_file)

    def forward(self, masked_ids, attention_mask, textf, adj, s_mask, lengths, labels, audio_ca):
        attention_mask = attention_mask.squeeze(0) #attention_mask多个句子就已经是批量了
        adj = adj.squeeze(0)
        s_mask = s_mask.squeeze(0)
        lengths = lengths.squeeze(0)

        audio_ca = self.audio_proj(audio_ca)
        context = self.context_gather(textf, adj, s_mask, lengths)

        ca = context.squeeze(0).unsqueeze(1) # 4 1 768
 
        ca = torch.cat((ca,audio_ca),dim = 1)
      
    #   M3EDdataset拼过了， 所以这里不用拼了， cped 没有拼
        # attention_mask = torch.cat((torch.unsqueeze(attention_mask[:, 0],1), torch.ones((ca.shape[0],ca.shape[1]), device=self.device), attention_mask[:, 1:]), 1).type(torch.long)
        attention_mask = torch.cat((torch.unsqueeze(attention_mask[:, 0],1),attention_mask[:, 1:]), 1).type(torch.long)

        token_type_ids = torch.zeros(masked_ids.shape, dtype=torch.long, device=self.device)
        # print('改造的attention_mask',attention_mask.shape)

        output = self.roberta_model(ca, masked_ids, attention_mask=attention_mask, token_type_ids=token_type_ids,labels=labels)
        return output
    
    def reload_weights(self, weights_file):
        model_dict = torch.load(weights_file)
        print("Reloading Model Weights")
        new_model_dict = {}
        # print('预训练的模型参数',model_dict.keys())
        for i,v in model_dict.items():
            if(i.find('context_gather') != -1):
                # print(i)
                new_model_dict[i] = v

        del new_model_dict['context_gather.fc1.weight']
        del new_model_dict['context_gather.fc1.bias']
        del new_model_dict['context_gather.attentive_node_features.transform.weight']
        del new_model_dict['context_gather.attentive_node_features.transform.bias']
        del new_model_dict['context_gather.attentive_node_features.fc.weight']

        # print(new_model_dict)
        # filter(self.tt,model_dict.items())
        self.load_state_dict(new_model_dict, strict=False)
        print("Model Weights Reloaded! :D")
    def tt(self,d):
        print(111)
        print(d[0],d[1])
        

# CPEDDataset

In [53]:
import numpy as np
# import warnings
# warnings.filterwarnings("ignore")
class CPEDDataset(torch.utils.data.Dataset):
    def __init__(self,  dataset_name = 'CPED', mode = 'train',device='cpu',config=None):
        path = './CPED_train_feature.pkl'
        if mode == 'val':
            path = './CPED_valid_feature.pkl'
        elif mode == 'test':
            path = './CPED_test_feature.pkl'

        self.videoIDs, self.videoSentence, self.videoSpeakers, self.videoLabels,self.textf = pickle.load(open(path, 'rb'))
        self.len = len(self.videoIDs)
        # self.len = 5
        self.tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-roberta-wwm-ext")
        self.label2idx = {'happy':0, 'grateful':1, 'relaxed':2, 'positive-other':3, 'neutral':4,'anger':5, 'sadness':6, 'fear':7, 'depress':8, 'disgust':9, 'astonished':10, 'worried': 11, 'negative-other':12}

    def __getitem__(self, index):
        vid = self.videoIDs[index]
        sentence = self.videoSentence[vid]
        max_len = len(sentence)
        # print(max_len)
        lengths = torch.tensor(max_len).unsqueeze(0)
        # print('lengths',lengths)
        adj = self.get_adj_v1([self.videoSpeakers[vid]], max_len)
        s_mask,_ = self.get_s_mask([self.videoSpeakers[vid]], max_len)
        # print('mask',s_mask)
        tokenizer_out= self.tokenizer(sentence, return_attention_mask=True, return_tensors="pt", padding=True, max_length=512, truncation=True)
        input_ids = tokenizer_out['input_ids'].to(device)
        attention_mask = tokenizer_out['attention_mask'].to(device)
        token_type_ids = tokenizer_out['token_type_ids'].to(device)
        textf = torch.tensor(self.textf[vid]).to(device)
        # print('textf',textf) # 基本都是0不知道为什么，待排查
        labels =[self.label2idx[label] for label in  self.videoLabels[vid]]
        return self.videoSentence[vid],\
                input_ids,\
                attention_mask,\
                token_type_ids,\
                torch.LongTensor(labels).to(device),\
                vid,\
                textf,\
                adj.to(device),\
                s_mask.to(device),\
                lengths.to(device)



    def get_adj_v1(self, speakers, max_dialog_len):
        '''
        get adj matrix
        :param speakers:  (B, N)
        :param max_dialog_len:
        :return:
            adj: (B, N, N) adj[:,i,:] means the direct predecessors of node i
        '''
        adj = []
        windowp = 2
        for speaker in speakers:
            a = torch.zeros(max_dialog_len, max_dialog_len)
            for i,s in enumerate(speaker):
                cnt = 0
                for j in range(i - 1, -1, -1):
                    a[i,j] = 1
                    if speaker[j] == s:
                        cnt += 1
                        if cnt==windowp:
                            break
            adj.append(a)
        return torch.stack(adj)
    
    def get_s_mask(self, speakers, max_dialog_len):
        '''
        :param speakers:
        :param max_dialog_len:
        :return:
         s_mask: (B, N, N) s_mask[:,i,:] means the speaker informations for predecessors of node i, where 1 denotes the same speaker, 0 denotes the different speaker
         s_mask_onehot (B, N, N, 2) onehot emcoding of s_mask
         一个完整对话中 和该句说话人相同的标记为1 , 不同的标记为0, s_mask_onehot 没有用到
        '''
        s_mask = []
        s_mask_onehot = []
        for speaker in speakers:
            s = torch.zeros(max_dialog_len, max_dialog_len, dtype = torch.long)
            s_onehot = torch.zeros(max_dialog_len, max_dialog_len, 2)
            for i in range(len(speaker)):
                for j in range(len(speaker)):
                    if speaker[i] == speaker[j]:
                        s[i,j] = 1
                        s_onehot[i,j,1] = 1
                    else:
                        s_onehot[i,j,0] = 1

            s_mask.append(s)
            s_mask_onehot.append(s_onehot)
        return torch.stack(s_mask), torch.stack(s_mask_onehot)

       
    def __len__(self):
        return self.len
######################测试dataset#################################
# device = torch.device('cuda')
# trainset = CPEDDataset(device=device)
# roberta_model = robertGatPretain(device,args)
# print(roberta_model.context_gather)
# train_loader = DataLoader(trainset,batch_size=1)
# for data in train_loader:
#     # print(data)
#     # break
#     sentences,input_ids,attention_mask,token_type_ids,labels,vid,textf,adj,s_mask,lengths = data
#     adj = adj.squeeze(0)
#     s_mask = s_mask.squeeze(0)
#     lengths = lengths.squeeze(0)
#     input_ids = input_ids.squeeze(0)
#     labels = labels.squeeze(0)
#     out = roberta_model(input_ids, attention_mask, textf,adj,s_mask,lengths,labels)
#     # print(out.loss)

# fetch_model_optim_sched_for_CPED

In [54]:
def fetch_model_optim_sched_for_CPED(config, device):
    model = robertGatPretain(device,args)
    # param_optimizer = list(model.named_parameters())
    # no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    # 为什么这些不衰减
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    # 为什么这些不衰减
    optimizer_grouped_parameters = [
        {
            "params": [
                p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.01,
        },
        {
            "params": [
                p for n, p in param_optimizer if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]

    # optimizer = AdamW(optimizer_grouped_parameters, lr=config['learning_rate'])
    optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)


    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=config['warmup_proportion'] * config['num_train_optimization_steps'],
        num_training_steps=config['num_train_optimization_steps'],
    )

    return model, optimizer,scheduler
# fetch_model_optim_sched(config, device)

# train_loop_for_CPED

In [55]:
def train_loop_for_CPED(train_dataloader, model, optimizer, scheduler, device, config):
    training_loss = 0
    gt_sentiment_scores = None
    pred_scores = None
    model.train()
    start_time = time.monotonic()
    epoch_preds = []
    epoch_labels = []
    for step, batch in enumerate(tqdm(train_dataloader)):
        sentences,input_ids,attention_mask,token_type_ids,labels,vid,textf,adj,s_mask,lengths = batch
        adj = adj.squeeze(0)
        s_mask = s_mask.squeeze(0)
        lengths = lengths.squeeze(0)
        input_ids = input_ids.squeeze(0)
        labels = labels.squeeze(0)
        output = model(input_ids, attention_mask, textf,adj,s_mask,lengths,labels)
        loss = output.loss

        if config['gradient_accumulation_step'] > 1:
            # loss = loss / config['gradient_accumulation_step']
            loss = loss / len(labels)

        loss.backward()

        training_loss += output.loss.item()

        if (step + 1) % config['gradient_accumulation_step'] == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        pred_scores = output.logits.detach().to(torch.device('cpu'))
        pred = torch.argmax(pred_scores,1).cpu().numpy().tolist()
        epoch_preds += pred
        
        label = labels.cpu().numpy().tolist()
        epoch_labels += label

        
    print("Training Batch time:", time.monotonic()-start_time)
    return training_loss / len(train_dataloader), epoch_preds, epoch_labels
def validation_loop_for_CPED(val_dataloader, model, device, config):
    val_loss = 0
    gt_sentiment_scores = None
    pred_scores = None
    model.eval()
    with torch.no_grad():
        epoch_preds = []
        epoch_labels = []
        for step, batch in enumerate(tqdm(val_dataloader)):
            sentences,input_ids,attention_mask,token_type_ids,labels,vid,textf,adj,s_mask,lengths = batch
            adj = adj.squeeze(0)
            s_mask = s_mask.squeeze(0)
            lengths = lengths.squeeze(0)
            input_ids = input_ids.squeeze(0)
            labels = labels.squeeze(0)
            output = model(input_ids, attention_mask, textf,adj,s_mask,lengths,labels)
            loss = output.loss

            if config['gradient_accumulation_step'] > 1:
                # loss = loss / config['gradient_accumulation_step']
                loss = loss / len(labels)


            val_loss += output.loss.item()
            pred_scores = output.logits.detach().to(torch.device('cpu'))
            pred = torch.argmax(pred_scores,1).cpu().numpy().tolist()
            epoch_preds += pred
            
            label = labels.cpu().numpy().tolist()
            epoch_labels += label
    
    return val_loss / len(val_dataloader),  epoch_preds, epoch_labels

In [56]:
# if __name__ == "__main__":
#     # RobertaConfig {
#     #   "attention_probs_dropout_prob": 0.1,
#     #   "bos_token_id": 0,
#     #   "classifier_dropout": null,
#     #   "eos_token_id": 2,
#     #   "hidden_act": "gelu",
#     #   "hidden_dropout_prob": 0.1,
#     #   "hidden_size": 768,
#     #   "initializer_range": 0.02,
#     #   "intermediate_size": 3072,
#     #   "layer_norm_eps": 1e-12,
#     #   "max_position_embeddings": 512,
#     #   "model_type": "roberta",
#     #   "num_attention_heads": 12,
#     #   "num_hidden_layers": 12,
#     #   "pad_token_id": 1,
#     #   "position_embedding_type": "absolute",
#     #   "transformers_version": "4.16.1",
#     #   "type_vocab_size": 2,
#     #   "use_cache": true,
#     #   "vocab_size": 30522
#     # }
#     roberta_config = transformers.RobertaConfig()
#     roberta_config.max_position_embeddings = 100
#     roberta_config.vocab_size = 21128
#     roberta_config.type_vocab_size = 1
#     roberta_config.num_labels = 7
#     # 微调的时候 是新训练了一个分类头
#     tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-roberta-wwm-ext")
#     roberta_model = RobertaForSequenceClassification.from_pretrained('hfl/chinese-roberta-wwm-ext', config=roberta_config).to(device)
#     # print(roberta_model)
# #

In [57]:
# ca = torch.rand((2, 1, 768), dtype=torch.float32).to(device)
# sentence = ['你不要生气了好不好','今天天气很好']
# tokenizer_out= tokenizer(sentence, return_attention_mask=True, return_tensors="pt", padding=True)
# print(tokenizer_out)

In [58]:
# input_ids = tokenizer_out['input_ids'].to(device)
# attention_mask = tokenizer_out['attention_mask'].to(device)
# token_type_ids = tokenizer_out['token_type_ids'].to(device)
# labels = torch.tensor([1,2]).to(device)
# ## <Create the input to maskedlm model> even adjust the attention_mask
# attention_mask = torch.cat((torch.unsqueeze(attention_mask[:, 0], 1), torch.ones((ca.shape[0], ca.shape[1]), device='cuda'), attention_mask[:, 1:]), 1).type(torch.long)
# token_type_ids = torch.zeros(input_ids.shape, dtype=torch.long, device='cuda')
# # print(input_ids,attention_mask,token_type_ids)
# final_output = roberta_model(ca, input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids,labels=labels)
# print(final_output)


# M3EDDataset

In [59]:
import numpy as np
class M3EDDataset(torch.utils.data.Dataset):
    def __init__(self,  dataset_name = 'M3ED', mode = 'train',device='cpu',config=None):
        path = '/usr/shixiaoxiao/mavis-datasets/M3ED/M3ED_features_files/Asent_wav2vec_zh2chmed2e5last-Vsent_avg_affectdenseface-Lsent_avg_robert_base_wwm_chinese.pkl'
        self.videoIDs, self.videoSpeakers, self.videoLabels, self.videoAudio, \
        self.videoText, self.videoVisual, self.videoSentence, self.trainVid, \
        self.valVid, self.testVid = pickle.load(open(path, 'rb'))
        '''
        label_to_idx = ['Happy', 'Neutral', 'Sad', 'Disgust', 'Anger', 'Fear', 'Surprise']
        idx_to_label = {'Happy':0, 'Neutral':1, 'Sad':2, 'Disgust':3, 'Anger': 4, 'Fear': 5, 'Surprise':6}
        '''
        self.keys = [x for x in (self.trainVid if mode=='train' else self.testVid if mode=='test' else self.valVid)]

        self.len = len(self.keys)
        # self.len = 5
        # self.tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-roberta-wwm-ext")
        
        self.tokenizer = AutoTokenizer.from_pretrained("cyclone/simcse-chinese-roberta-wwm-ext") 
        self.max_text_length = config['MAX_TEXT_LENGTH']
    def __getitem__(self, index):
        vid = self.keys[index]
        speakers = self.videoSpeakers[vid]
        windowsp = 1
        arr = [''] * len(speakers)
        for k in range(len(speakers)-1,-1,-1):
            # if k > 2 and (speakers[k-3] == speakers[k-2]) and (speakers[k-2] == speakers[k-1]) and (speakers[k-1] == speakers[k]):
            #     self.videoSentence[vid][k] = self.videoSentence[vid][k-3]  + self.videoSentence[vid][k-2] + self.videoSentence[vid][k-1] + self.videoSentence[vid][k]
            # elif k > 1 and (speakers[k-2] == speakers[k-1]) and (speakers[k-1] == speakers[k]):
            #     self.videoSentence[vid][k] = self.videoSentence[vid][k-2] + self.videoSentence[vid][k-1] + self.videoSentence[vid][k] 
            # elif  k > 0 and  (speakers[k-1] == speakers[k]):
            #     self.videoSentence[vid][k] = self.videoSentence[vid][k-1] + self.videoSentence[vid][k] 
            if k > 1 and (speakers[k-2] == speakers[k-1]) and (speakers[k-1] == speakers[k]):
                arr[k] = str(self.videoSentence[vid][k-2] )+ str(self.videoSentence[vid][k-1] )+str(self.videoSentence[vid][k] ) 
            elif  k > 0 and  (speakers[k-1] == speakers[k]):
                arr[k] =str(self.videoSentence[vid][k-1])+str(self.videoSentence[vid][k] )
            elif k >= 0:
                arr[k] = str(self.videoSentence[vid][k])

        # 原来 直接合并的时候文本会被截断
        tokenizer_out= self.tokenizer(arr, return_attention_mask=True, return_tensors="pt", padding=True, max_length=512, truncation=True)
        input_ids = tokenizer_out['input_ids'].to(device)
        attention_mask = tokenizer_out['attention_mask'].to(device)
        token_type_ids = tokenizer_out['token_type_ids'].to(device)
        ca = self.videoAudio[vid]
        ca = torch.from_numpy(ca).unsqueeze(1).to(device)
        attention_mask = torch.cat((torch.unsqueeze(attention_mask[:, 0], 1), torch.ones((ca.shape[0], ca.shape[1] * 2), device='cuda'),  attention_mask[:, 1:]), 1).type(torch.long)
        labels = torch.tensor(self.videoLabels[vid]).to(device)
        textf = torch.tensor(self.videoText[vid]).to(device)
        adj = self.get_adj_v1([speakers],len(speakers)).to(device)
        s_mask,_ = self.get_s_mask([speakers],len(speakers))
        lengths = torch.tensor(len(speakers)).unsqueeze(0).to(device)

        return ca, input_ids, attention_mask, token_type_ids, labels, textf, adj, s_mask.to(device), lengths
    
    def get_adj_v1(self, speakers, max_dialog_len):
        '''
        get adj matrix
        :param speakers:  (B, N)
        :param max_dialog_len:
        :return:
            adj: (B, N, N) adj[:,i,:] means the direct predecessors of node i
        '''
        adj = []
        windowp = 2
        for speaker in speakers:
            a = torch.zeros(max_dialog_len, max_dialog_len)
            for i,s in enumerate(speaker):
                cnt = 0
                for j in range(i - 1, -1, -1):
                    a[i,j] = 1
                    if speaker[j] == s:
                        cnt += 1
                        if cnt==windowp:
                            break
            adj.append(a)
        return torch.stack(adj)
    
    def get_s_mask(self, speakers, max_dialog_len):
        '''
        :param speakers:
        :param max_dialog_len:
        :return:
         s_mask: (B, N, N) s_mask[:,i,:] means the speaker informations for predecessors of node i, where 1 denotes the same speaker, 0 denotes the different speaker
         s_mask_onehot (B, N, N, 2) onehot emcoding of s_mask
         一个完整对话中 和该句说话人相同的标记为1 , 不同的标记为0, s_mask_onehot 没有用到
        '''
        s_mask = []
        s_mask_onehot = []
        for speaker in speakers:
            s = torch.zeros(max_dialog_len, max_dialog_len, dtype = torch.long)
            s_onehot = torch.zeros(max_dialog_len, max_dialog_len, 2)
            for i in range(len(speaker)):
                for j in range(len(speaker)):
                    if speaker[i] == speaker[j]:
                        s[i,j] = 1
                        s_onehot[i,j,1] = 1
                    else:
                        s_onehot[i,j,0] = 1

            s_mask.append(s)
            s_mask_onehot.append(s_onehot)
        return torch.stack(s_mask), torch.stack(s_mask_onehot)
    
    def __len__(self):
        return self.len
        
######################测试dataset#################################
# trainset = M3EDDataset(device=device, config=config)

# train_loader = DataLoader(trainset,batch_size=1)
# for data in train_loader:
#     # ca, input_ids, attention_mask, token_type_ids, labels = [d.squeeze(0) for d in data]
#     # out = roberta_model(ca, input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids,labels=labels)
#     # print(out.loss)
#     pass

# IEMOCAPDataset

In [60]:
class IEMOCAPDataset(torch.utils.data.Dataset):

    def __init__(self, train=True):
        path = '/usr/shixiaoxiao/mavis-project/EmoCaps/iemocap_data.pkl'
        self.videoIDs, self.videoSpeakers, self.videoLabels, self.videoText,\
        self.videoAudio, self.videoVisual, self.videoSentence, self.trainVid,\
        self.testVid = pickle.load(open(path, 'rb'), encoding="latin1")
        '''
        label index mapping = {'hap':0, 'sad':1, 'neu':2, 'ang':3, 'exc':4, 'fru':5}
        '''
        del self.videoVisual['Ses05F_script02_2']
        del self.videoAudio['Ses05F_script02_2']
        del self.videoSpeakers['Ses05F_script02_2']
        del self.videoLabels['Ses05F_script02_2']
        self.testVid.remove('Ses05F_script02_2')
        self.keys = [x for x in (self.trainVid if train else self.testVid)]
        self.len = len(self.keys)
        # self.len = 5
        self.tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-roberta-base") 

    def __getitem__(self, index):
        vid = self.keys[index]
        speakers = self.videoSpeakers[vid]
        windowsp = 1
        arr = [''] * len(speakers)
        for k in range(len(speakers)-1,-1,-1):
            # if k > 2 and (speakers[k-3] == speakers[k-2]) and (speakers[k-2] == speakers[k-1]) and (speakers[k-1] == speakers[k]):
            #     self.videoSentence[vid][k] = self.videoSentence[vid][k-3]  + self.videoSentence[vid][k-2] + self.videoSentence[vid][k-1] + self.videoSentence[vid][k]
            # elif k > 1 and (speakers[k-2] == speakers[k-1]) and (speakers[k-1] == speakers[k]):
            #     self.videoSentence[vid][k] = self.videoSentence[vid][k-2] + self.videoSentence[vid][k-1] + self.videoSentence[vid][k] 
            # elif  k > 0 and  (speakers[k-1] == speakers[k]):
            #     self.videoSentence[vid][k] = self.videoSentence[vid][k-1] + self.videoSentence[vid][k] 
            if k > 1 and (speakers[k-2] == speakers[k-1]) and (speakers[k-1] == speakers[k]):
                arr[k] = str(self.videoSentence[vid][k-2] )+ str(self.videoSentence[vid][k-1] )+str(self.videoSentence[vid][k] ) 
            elif  k > 0 and  (speakers[k-1] == speakers[k]):
                arr[k] =str(self.videoSentence[vid][k-1])+str(self.videoSentence[vid][k] )
            elif k >= 0:
                arr[k] = str(self.videoSentence[vid][k])

        # 原来 直接合并的时候文本会被截断
        tokenizer_out= self.tokenizer(arr, return_attention_mask=True, return_tensors="pt", padding=True, max_length=512, truncation=True)
        input_ids = tokenizer_out['input_ids'].to(device)
        attention_mask = tokenizer_out['attention_mask'].to(device)
        # token_type_ids = tokenizer_out['token_type_ids'].to(device)
        ca = self.videoAudio[vid]
     
        ca = torch.tensor(ca).unsqueeze(1).to(device)
        attention_mask = torch.cat((torch.unsqueeze(attention_mask[:, 0], 1), torch.ones((ca.shape[0], ca.shape[1] * 2), device='cuda'),  attention_mask[:, 1:]), 1).type(torch.long)
        labels = torch.tensor(self.videoLabels[vid]).to(device)
        textf = torch.from_numpy(self.videoText[vid]).float().to(device)
        adj = self.get_adj_v1([speakers],len(speakers)).to(device)
        s_mask,_ = self.get_s_mask([speakers],len(speakers))
        lengths = torch.tensor(len(speakers)).unsqueeze(0).to(device)
        # t 100 a 100 v 256
        # print('ca',ca.shape,ca.dtype)
        # print('input_ids',input_ids.shape,input_ids.dtype)
        # print('attention_mask',attention_mask.shape,attention_mask.dtype)
        # print('token_type_ids',token_type_ids.shape,token_type_ids.dtype)
        # print('labels',labels.shape,labels.dtype)
        # print('textf',textf.shape,textf.dtype)
        # print('adj',adj.shape,adj.dtype)
        # print('s_mask',s_mask.shape,s_mask.dtype)
        # print('lengths',lengths.shape,lengths.dtype)
        return ca, input_ids, attention_mask,  labels, textf, adj, s_mask.to(device), lengths
    
    def get_adj_v1(self, speakers, max_dialog_len):
        '''
        get adj matrix
        :param speakers:  (B, N)
        :param max_dialog_len:
        :return:
            adj: (B, N, N) adj[:,i,:] means the direct predecessors of node i
        '''
        adj = []
        windowp = 2
        for speaker in speakers:
            a = torch.zeros(max_dialog_len, max_dialog_len)
            for i,s in enumerate(speaker):
                cnt = 0
                for j in range(i - 1, -1, -1):
                    a[i,j] = 1
                    if speaker[j] == s:
                        cnt += 1
                        if cnt==windowp:
                            break
            adj.append(a)
        return torch.stack(adj)
    
    def get_s_mask(self, speakers, max_dialog_len):
        '''
        :param speakers:
        :param max_dialog_len:
        :return:
         s_mask: (B, N, N) s_mask[:,i,:] means the speaker informations for predecessors of node i, where 1 denotes the same speaker, 0 denotes the different speaker
         s_mask_onehot (B, N, N, 2) onehot emcoding of s_mask
         一个完整对话中 和该句说话人相同的标记为1 , 不同的标记为0, s_mask_onehot 没有用到
        '''
        s_mask = []
        s_mask_onehot = []
        for speaker in speakers:
            s = torch.zeros(max_dialog_len, max_dialog_len, dtype = torch.long)
            s_onehot = torch.zeros(max_dialog_len, max_dialog_len, 2)
            for i in range(len(speaker)):
                for j in range(len(speaker)):
                    if speaker[i] == speaker[j]:
                        s[i,j] = 1
                        s_onehot[i,j,1] = 1
                    else:
                        s_onehot[i,j,0] = 1

            s_mask.append(s)
            s_mask_onehot.append(s_onehot)
        return torch.stack(s_mask), torch.stack(s_mask_onehot)
    
    def __len__(self):
        return self.len
# trainset  = IEMOCAPDataset()
# loader = DataLoader(trainset,batch_size=1)
# for data in loader:
#     print(data)

In [61]:
def read_config(config_file, args):
    if not os.path.exists(config_file):
        raise FileNotFoundError(config_file)
    
    loader = yaml.SafeLoader
    loader.add_implicit_resolver(
        u'tag:yaml.org,2002:float',
        re.compile(u'''^(?:
         [-+]?(?:[0-9][0-9_]*)\\.[0-9_]*(?:[eE][-+]?[0-9]+)?
        |[-+]?(?:[0-9][0-9_]*)(?:[eE][-+]?[0-9]+)
        |\\.[0-9_]+(?:[eE][-+][0-9]+)?
        |[-+]?[0-9][0-9_]*(?::[0-5]?[0-9])+\\.[0-9_]*
        |[-+]?\\.(?:inf|Inf|INF)
        |\\.(?:nan|NaN|NAN))$''', re.X),
        list(u'-+0123456789.'))

    with open(config_file, 'r') as f:
        config = yaml.load(f, Loader=loader)
    
    ## Override Args Values
    config['num_epochs'] = args.num_epochs
    config['MAX_TEXT_LENGTH'] = args.max_text_length
    config['MAX_TIME'] = args.max_time
    config['learning_rate'] = args.learning_rate
    config['warmup_proportion'] = args.warmup_proportion
    config['batch_size'] = args.batch_size
    
    config['MAX_AUDIO_LENGTH'] = config['MAX_TIME'] * config['SAMPLING_RATE']
    config['gradient_accumulation_step'] = 8 # 不确定

    return config
config = read_config('./config_mosi.yaml',args)
print(config)

{'seed': 42, 'SAMPLING_RATE': 16000, 'MAX_TIME': 30, 'MAX_TEXT_LENGTH': 150, 'device': 'cuda', 'mode': 'training', 'pretrained_model_file': '/usr/shixiaoxiao/mavis-project/TEASEL/artificats/04212022_203118/model_ep_9.pt', 'batch_size': 1, 'num_workers': 2, 'learning_rate': 2e-05, 'num_epochs': 80, 'gradient_accumulation_step': 8, 'warmup_proportion': 0.1, 'model_output_dir': 'artificats/', 'MAX_AUDIO_LENGTH': 480000}


# fetch_model_optim_sched_for_IEOCOMP

In [62]:
def set_random_seeds(seed):
    print("Seed: {}".format(seed))

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.deterministic = True

    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def fetch_model_optim_sched(config, device, args):
    model = robertGatFinetune(device, args)

    param_optimizer = list(model.named_parameters())
    # for k ,v in param_optimizer:
    #     print(k)
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    # tmp =  {n:p for n, p in param_optimizer if not any(nd in n for nd in no_decay)}
    # for k ,v in tmp.items():
    #     print(k)    
    # tmp =  {n:p for n, p in param_optimizer if  any(nd in n for nd in no_decay)}
    # for k ,v in tmp.items():
    #     print(k)  
    
    # 为什么这些不衰减
    optimizer_grouped_parameters = [
        # {
        #     "params": [
        #         p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
        #     ],
        #     "weight_decay": 0.01,
        # },
        # {
        #     "params": [
        #         p for n, p in param_optimizer if any(nd in n for nd in no_decay)
        #     ],
        #     "weight_decay": 0.0,
        # },
        {
            'params':[
                p for n, p in param_optimizer if any(nd in n for nd in ['context_gather', 'audio_proj'])
            ],
            "lr": 1e-4,
            "weight_decay": 0.01,
        },
        # {
        #     'params':[
        #         p for n, p in param_optimizer if not any(nd in n for nd in ['context_gather', 'audio_proj'])

        #     ],
        #     "lr": 0.0,
        #     "weight_decay": 0.00,
        # }
    ]
    parameter_ids = [[id(p) for p in group["params"]] for group in optimizer_grouped_parameters]


    # optimizer = AdamW(optimizer_grouped_parameters, lr=config['learning_rate'])
    # optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=2e-5)
    optimizer = AdamW(optimizer_grouped_parameters)


    
    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=config['warmup_proportion'] * config['num_train_optimization_steps'],
        num_training_steps=config['num_train_optimization_steps'],
    )

    return model, optimizer, scheduler
# fetch_model_optim_sched(config, device)

# train_loop

In [63]:

def train_loop(train_dataloader, model, optimizer, scheduler, device, config):
    training_loss = 0
    gt_sentiment_scores = None
    pred_scores = None
    model.train()
    start_time = time.monotonic()
    epoch_preds = []
    epoch_labels = []
    for step, batch in enumerate(tqdm(train_dataloader)):

        audio_ca , input_ids, attention_mask,  labels ,  textf, adj, s_mask, lengths= [d.squeeze(0) if idx  < 4 else d for idx, d in enumerate(batch) ]

        output = model(input_ids, attention_mask, textf, adj, s_mask, lengths, labels, audio_ca)
        loss = output.loss

        if config['gradient_accumulation_step'] > 1:
            # loss = loss / config['gradient_accumulation_step']
            loss = loss / len(labels)

        loss.backward()

        training_loss += output.loss.item()

        if (step + 1) % config['gradient_accumulation_step'] == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        pred_scores = output.logits.detach().to(torch.device('cpu'))
        pred = torch.argmax(pred_scores,1).cpu().numpy().tolist()
        epoch_preds += pred
        
        label = labels.cpu().numpy().tolist()
        epoch_labels += label

        # if pred_scores is None:
        #     pred_scores = output.logits.detach().to(torch.device('cpu'))
        #     # gt_sentiment_scores = gt_scores.detach().to(torch.device('cpu'))  # 指ground truth
        #     true_labels = labels.detach().to(torch.device('cpu'))
        # else:
        #     pred_scores = torch.vstack((pred_scores, output.logits.detach().to(torch.device('cpu')))).to(torch.device('cpu'))
        #     # gt_sentiment_scores = torch.vstack((gt_sentiment_scores, gt_scores.detach().to(torch.device('cpu')))).to(torch.device('cpu'))
        #     true_labels = torch.vstack((true_labels, labels.detach().to(torch.device('cpu')))).to(torch.device('cpu'))

    print("Training Batch time:", time.monotonic()-start_time)
    
    return training_loss / len(train_dataloader), epoch_preds, epoch_labels

def validation_loop(val_dataloader, model, device, config):
    val_loss = 0
    gt_sentiment_scores = None
    pred_scores = None
    model.eval()
    with torch.no_grad():
        epoch_preds = []
        epoch_labels = []
        for step, batch in enumerate(tqdm(val_dataloader)):
            audio_ca , input_ids, attention_mask,  labels ,  textf, adj, s_mask, lengths= [d.squeeze(0) if idx  < 4 else d for idx, d in enumerate(batch) ]

            output = model(input_ids, attention_mask, textf, adj, s_mask, lengths, labels, audio_ca)
            loss = output.loss

            if config['gradient_accumulation_step'] > 1:
                # loss = loss / config['gradient_accumulation_step']
                loss = loss / len(labels)


            val_loss += output.loss.item()

            # if pred_scores is None:
            #     pred_scores = output.logits.detach().to(torch.device('cpu'))
            #     # gt_sentiment_scores = gt_scores.detach().to(torch.device('cpu'))
            # else:
            #     pred_scores = torch.vstack((pred_scores, output.logits.detach().to(torch.device('cpu')))).to(torch.device('cpu'))
            #     # gt_sentiment_scores = torch.vstack((gt_sentiment_scores, gt_scores.detach().to(torch.device('cpu')))).to(torch.device('cpu'))
            pred_scores = output.logits.detach().to(torch.device('cpu'))
            pred = torch.argmax(pred_scores,1).cpu().numpy().tolist()
            epoch_preds += pred
            
            label = labels.cpu().numpy().tolist()
            epoch_labels += label
    
    return val_loss / len(val_dataloader),  epoch_preds, epoch_labels

def compute_metrics(pred_scores, gt_sentiment_scores):
    pred_scores = pred_scores.flatten().detach().numpy()
    gt_sentiment_scores = gt_sentiment_scores.flatten().detach().numpy()
    
    mae = mean_absolute_error(y_true=gt_sentiment_scores, y_pred=pred_scores)
    binary_acc = accuracy_score(y_true=gt_sentiment_scores>0, y_pred=pred_scores>0)
    f1 = f1_score(y_true=gt_sentiment_scores>0, y_pred=pred_scores>0)
    corr = np.corrcoef(pred_scores, gt_sentiment_scores)[0][1]
    
    return mae, binary_acc, f1, corr


# main

In [64]:

## Create Wandb object
config = read_config('config_mosi.yaml', args)
# Set All Random Seeds
# set_random_seeds(456)

##Device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Device: ",device)

##Create Output Directory if it doesn't exists
# config['model_internal_dir'] = datetime.datetime.now().strftime('%m%d%Y_%H%M%S')
# if not os.path.exists(config['model_output_dir']):
#     os.mkdir(config['model_output_dir'])

# os.mkdir(Path(config['model_output_dir']) / config['model_internal_dir'])
# 
# print("Model Dir: ", Path(config['model_output_dir']) / config['model_internal_dir'])

# Create All Data Loaders
print("Preparing Datasets and DataLoaders")

train_dataset = IEMOCAPDataset(train = True)
test_dataset = IEMOCAPDataset(train = False)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=config['batch_size'])
print("Datasets and DataLoaders Done! :D")

config['num_train_optimization_steps'] = (int(len(train_dataset) / config['batch_size'] / config['gradient_accumulation_step']) * config['num_epochs'])

print("Creating Model")
model, optimizer, scheduler = fetch_model_optim_sched(config, device,args)
print("Model Created! :D")


Device:  cuda
Preparing Datasets and DataLoaders
Datasets and DataLoaders Done! :D
Creating Model


Some weights of the model checkpoint at princeton-nlp/sup-simcse-roberta-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at princeton-nlp/sup-simcse-roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model 

Model Created! :D


In [65]:

# ## Create Wandb object
# # config = read_config('config_mosi.yaml', args)
# ## Set All Random Seeds
# # set_random_seeds(config['seed'])

# ##Device
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# print("Device: ",device)

# print("Preparing Datasets and DataLoaders")

# train_dataset = CPEDDataset(mode='train', device=device, config=config)
# val_dataset = CPEDDataset(mode='val', device=device, config=config)
# test_dataset = CPEDDataset(mode='test', device=device, config=config)

# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
# val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=config['batch_size'])
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=config['batch_size'])
# print("Datasets and DataLoaders Done! :D")

# config['num_train_optimization_steps'] = (int(len(train_dataset) / config['batch_size'] / config['gradient_accumulation_step']) * config['num_epochs'])

# print("Creating Model")
# model, optimizer, scheduler = fetch_model_optim_sched_for_CPED(config, device)
# print("Model Created! :D")


# 开始训练

In [66]:
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

print("Starting Training")
best_val_loss = None
# wandb.init(config=args,\
#            project='5-27 roberta_fine_tune_SIMCSE_IEOCOMP_锁定.ipynb',\
#             notes='TEOCAMP结果')

for epoch in range(config['num_epochs']):
    print(f"Epoch: {epoch}")
    tloss, train_epoch_preds, train_epoch_labels = train_loop(train_loader, model, optimizer, scheduler, device, config)
    print('--------------------train--------------')
    print(classification_report(train_epoch_labels,train_epoch_preds,digits=4))
    # vloss, val_epoch_preds,val_epoch_labels= validation_loop(val_loader, model, device, config)
   
    # print('--------------------val--------------')
    # print(classification_report(val_epoch_labels,val_epoch_preds))
    testloss, test_epoch_preds,test_epoch_labels= validation_loop(test_loader, model, device, config)
    if best_val_loss is None:
        best_val_loss = testloss
    elif best_val_loss > testloss:
        best_val_loss = testloss
        print('testloss loss 下降了↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓')
        PATH = './best_IEMCOAP_freeze_bert.ckt'
        print('正在保存模型',PATH)
        torch.save(model.state_dict(), PATH)
    print('--------------------test--------------')

    print(classification_report(test_epoch_labels,test_epoch_preds,digits=4))
    
    # wandb.log({'train_loss':tloss, 'test_loss': testloss})
    # wandb.log(classification_report(train_epoch_labels,train_epoch_preds,digits=4,output_dict=True))
    # wandb.log(classification_report(test_epoch_labels,test_epoch_preds,digits=4,output_dict=True,target_names=['hap', 'sad', 'neu', 'ang', 'exc', 'fru']))
    # wandb.log(classification_report(test_epoch_labels,test_epoch_preds,digits=4,output_dict=True))
    # print(tloss,'|', '|', testloss)
    
print("Finished Training! :D")
# wandb.finish(0)

Starting Training
Epoch: 0


  0%|          | 0/120 [00:00<?, ?it/s]

Training Batch time: 39.46402762700018
--------------------train--------------
              precision    recall  f1-score   support

           0     0.0404    0.0218    0.0284       504
           1     0.1516    0.2098    0.1760       839
           2     0.2254    0.5634    0.3220      1324
           3     0.1605    0.1115    0.1316       933
           4     0.1667    0.0013    0.0027       742
           5     0.2827    0.0634    0.1035      1468
           6     0.0000    0.0000    0.0000         0

    accuracy                         0.1947      5810
   macro avg     0.1468    0.1387    0.1092      5810
weighted avg     0.1953    0.1947    0.1489      5810



  0%|          | 0/30 [00:00<?, ?it/s]

--------------------test--------------
              precision    recall  f1-score   support

           0     0.0455    0.0074    0.0127       135
           1     0.1928    0.2795    0.2282       229
           2     0.2436    0.6755    0.3580       379
           3     0.0800    0.0595    0.0683       168
           4     0.0000    0.0000    0.0000       295
           5     0.3636    0.0335    0.0614       358
           6     0.0000    0.0000    0.0000         0

    accuracy                         0.2193      1564
   macro avg     0.1322    0.1508    0.1041      1564
weighted avg     0.1830    0.2193    0.1427      1564

Epoch: 1


...

testloss loss 下降了↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
正在保存模型 ./best_IEMCOAP_freeze_bert.ckt
--------------------test--------------
              precision    recall  f1-score   support

           0     0.8276    0.7111    0.7649       135
           1     0.8279    0.8821    0.8541       229
           2     0.7947    0.7863    0.7905       379
           3     0.8321    0.6786    0.7475       168
           4     0.8791    0.8136    0.8451       295
           5     0.6802    0.7961    0.7336       358

    accuracy                         0.7896      1564
   macro avg     0.8069    0.7780    0.7893      1564
weighted avg     0.7961    0.7896    0.7902      1564

Epoch: 79


  0%|          | 0/120 [00:00<?, ?it/s]

Training Batch time: 39.323553963999075
--------------------train--------------
              precision    recall  f1-score   support

           0     0.9091    0.8730    0.8907       504
           1     0.9373    0.9273    0.9323       839
           2     0.8901    0.8927    0.8914      1324
           3     0.8833    0.8114    0.8458       933
           4     0.9319    0.8666    0.8980       742
           5     0.8267    0.9128    0.8676      1468

    accuracy                         0.8847      5810
   macro avg     0.8964    0.8806    0.8876      5810
weighted avg     0.8868    0.8847    0.8848      5810



  0%|          | 0/30 [00:00<?, ?it/s]

--------------------test--------------
              precision    recall  f1-score   support

           0     0.8276    0.7111    0.7649       135
           1     0.8279    0.8821    0.8541       229
           2     0.7947    0.7863    0.7905       379
           3     0.8321    0.6786    0.7475       168
           4     0.8791    0.8136    0.8451       295
           5     0.6802    0.7961    0.7336       358

    accuracy                         0.7896      1564
   macro avg     0.8069    0.7780    0.7893      1564
weighted avg     0.7961    0.7896    0.7902      1564

Finished Training! :D
